## Curve fiting of Cell Painting by Dose 

In order to start plotting the curves, we made some modifications to the dose values:  

- we transformed the doses to a logarithmic scale: `dose_log <- log10(dose)`  
- we set the dose spacing to 1: `dose_spacing = 1`  
- we shifted the dose values to ensure all were positive:  
  `dose_shift = abs(min(dose_log)) + dose_spacing`  
- finally, we applied the shift: `dose_log_shifted <- dose_log + dose_shift`  

We then used the **fastbmdR** package on our **norm_signal** data, after filtering out proteins that had 100% missing values.

In [1]:
#
library(ggplot2)

#install.packages("drc", dependencies = TRUE)

devtools::install_github("jessica-ewald/fastbmdR")


Using GitHub PAT from the git credential store.

Skipping install of 'fastbmdR' from a github remote, the SHA1 (b3681861) has not changed since last install.
  Use `force = TRUE` to force installation



In [ ]:
library(fastbmdR)
library(dplyr)
library(knitr)

## Cell Count From CP data

We add the count cell columns from CP data to the proteomic data according to the plate number and compound well. 

In [ ]:
library(arrow)
library(dplyr)

raw_data <- read_parquet("../00.exploratory_analysis/inputs/CP_data/raw.parquet")

raw_data <- raw_data %>% 
  filter(Metadata_BROAD_ID != "EMPTY")

raw_data <- raw_data %>%
  mutate(
    `Metadata_Compound Name` = ifelse(Metadata_BROAD_ID == "DMSO", "DMSO", `Metadata_Compound Name`),
    Metadata_Annotation = ifelse(Metadata_BROAD_ID == "DMSO", "DMSO", Metadata_Annotation),
    `Metadata_Compound Name` = ifelse(Metadata_BROAD_ID == "EMPTY", "EMPTY", `Metadata_Compound Name`),
    Metadata_Annotation = ifelse(Metadata_BROAD_ID == "EMPTY", "no treatment", Metadata_Annotation)
  )

selected_data <- raw_data %>%
  select('Metadata_Well', 'Metadata_Count_Cells', 'Metadata_plate_map_name')

In [2]:
# we add the cell count from CP data to Proteiomic imputed data + the probability of the RF classification

## all done no need to re-run this code just read the data df_CC_Proba.csv


df <- read.csv("../00.exploratory_analysis/outputs/norm_signal_filtered.csv", sep= ',')

df_pred <- read.csv("./outputs/df_pred.csv", sep= ',')


df <- df %>%
  mutate(Metadata_plate_map_name = case_when(
    Metadata_Plate == "BR00145683" ~ "BR00145695",
    Metadata_Plate == "BR00145686" ~ "BR00145692",
    TRUE ~ Metadata_Plate 
  ))


df <- df %>%
  left_join(selected_data, 
            by = c("Metadata_Well" = "Metadata_Well", 
                   "Metadata_plate_map_name" = "Metadata_plate_map_name"))


df <- merge(df, df_pred[, c("Metadata_Well", "Metadata_plate_map_name", "Probability")], 
            by = c("Metadata_Well", "Metadata_plate_map_name"),
            all.x = TRUE)

# Replacing the missing values probability with 0 from Jump controls and DMSO

df$Probability[is.na(df$Probability)] <- 0

sum(is.na(df$Probability))

#write.csv(df, file = "./outputs/df_CC_Proba.csv", row.names = FALSE)

[1] 0

In [118]:
#df <- read.csv("./outputs/df_CC_Proba.csv", sep= ',')


# we keep only the columns of interest

df_cc <- df[, c("Metadata_Compound", 'Probability', "Metadata_Concentration", "Metadata_Count_Cells")]


df_cc <- df_cc[df_cc$Metadata_Compound != "UNTREATED", ]


df_cc <- df_cc %>%
  rename(
    CC0 = Metadata_Count_Cells,
    RF_prob= Probability
  )


print(head(df_cc))


          Metadata_Compound RF_prob Metadata_Concentration  CC0
1                 LY2109761    0.00                  5.100 1570
3                Cladribine    0.50                  1.235 3059
4             Actinomycin D    0.61                300.000 1914
5              Treprostinil    0.55                  3.704 3355
6                 Bevirimat    0.61                  3.704 4112
7 Aminodarone Hydrochloride    0.51                  0.154 3784


### Curves of all compounds : Cell Count  vs Dose

In [ ]:
models <- c("Exp2", "Exp3", "Exp4", "Exp5", "Poly2", "Lin", "Power", "Hill")

ncpus <- 1  

unique_compounds <- unique(df_cc$Metadata_Compound)

feat_cols = colnames(df_cc)[!grepl("Metadata", colnames(df_cc))]


res = list()
gene_table <- list()

all_bmd_pass <- data.frame()
results_df <- data.frame()

for (compound in unique_compounds) {

    compound_data <- df_cc[df_cc$Metadata_Compound %in% c(compound, "DMSO"), ]
    dose <- compound_data$Metadata_Concentration
    dose_log <- dose
    dose_log[dose_log > 0] <- log10(dose_log[dose_log > 0])

    rank_dose <- unique(dose) %>% sort(decreasing = TRUE)
    dose_spacing <- abs(log10(rank_dose[2] / rank_dose[1]))
    dose_shift <- abs(min(dose_log)) + dose_spacing

    dose_shifted <- dose_log
    dose_shifted[dose_shifted != 0] <- dose_shifted[dose_shifted != 0] + dose_shift

    if (length(unique(compound_data$Metadata_Concentration)) > 2) {
        #cat("Processing compound:", compound, "\n")

        mat <- t(compound_data[, feat_cols])
        min_val <- abs(min(mat, na.rm = TRUE))
        mat_shifted <- mat + min_val + 0.1 * min_val

        if (length(dose_shifted) == ncol(mat_shifted)) {
            cat("Processing compound:", compound, "\n")
            tryCatch({
                
                fit <- PerformCurveFitting(data = mat_shifted, dose = dose_shifted, ncpus = ncpus, models = models)
                fit_filtered <- FilterDRFit(fit, lof.pval = 0.1, filt.var = "AIC.model")
                fit_final <- PerformBMDCalc(fit_filtered, ncpus = ncpus, num.sds = 2, bmr.method = "sample.mean", log10.dose = TRUE)

              
                bmd_pass <- fit_final$bmd_res
                bmd_pass <- bmd_pass[bmd_pass$gene.id== 'CC0', ]
            
                plot_data <- plot_bmd_curve('CC0', fit_final, return_type = "plot.data")
                plot_data$compound_name <- compound
        
                bmd_pass_valid <- bmd_pass[bmd_pass$all.pass == TRUE, c("bmd", "bmdl", "bmdu")]
      
                for (i in 1:nrow(bmd_pass_valid)) {
                    plot_data$bmd <- bmd_pass_valid$bmd[i]
                    plot_data$bmd_l <- bmd_pass_valid$bmdl[i]
                    plot_data$bmd_u <- bmd_pass_valid$bmdu[i]

        
                    results_df <- rbind(results_df, plot_data)
                }

            }, error = function(e) {
           
                message(sprintf("Error with compound %s: %s", compound, e$message))
            })
        }
    }
}

# Afficher les premiers résultats de results_df
print(head(resultss_df))


In [135]:
n_per_page <- 9  


plot_groups_CC0 <- unique(results_df$compound_name)
n_pages_CC0 <- ceiling(length(plot_groups_CC0) / n_per_page)


pdf_file_CC0 <- "./outputs/cc0_bmd_curves_all.pdf"
pdf(pdf_file_CC0, width = 15, height = 10)

for (i in 1:n_pages_CC0) {
  tryCatch({
    p <- ggplot(results_df, aes(x = x, y = Observations)) +
      geom_point(show.legend = FALSE) +
      geom_line(aes(y = f_x), show.legend = FALSE) +
      geom_vline(aes(xintercept = bmd), linetype = "solid", color = "red") +  
      geom_vline(aes(xintercept = bmd_l), linetype = "dashed", color = "red") + 
      geom_vline(aes(xintercept = bmd_u), linetype = "dashed", color = "red") +  
      facet_wrap_paginate(~compound_name, ncol = 3, nrow = 3, page = i, scales = "free_y") +
      labs(title = sprintf("BMD curves - CC0 - Page %d", i),
           x = "Concentration",
           y = "Observation") +
      theme_bw() +
      theme(strip.text = element_text(size = 8))
    
    print(p) 
  }, error = function(e) {
    message(sprintf("Error in the page %d : %s", i, e$message))
  })
}

dev.off()

Warning message:
"Removed 2450 rows containing missing values or values outside the scale range
(`geom_point()`)."
Warning message:
"Removed 4186 rows containing missing values or values outside the scale range
(`geom_vline()`)."
Warning message:
"Removed 4186 rows containing missing values or values outside the scale range
(`geom_vline()`)."
Warning message:
"Removed 4186 rows containing missing values or values outside the scale range
(`geom_vline()`)."
Warning message:
"Removed 2450 rows containing missing values or values outside the scale range
(`geom_point()`)."
Warning message:
"Removed 4186 rows containing missing values or values outside the scale range
(`geom_vline()`)."
Warning message:
"Removed 4186 rows containing missing values or values outside the scale range
(`geom_vline()`)."
Warning message:
"Removed 4186 rows containing missing values or values outside the scale range
(`geom_vline()`)."
Warning message:
"Removed 2450 rows containing missing values or values outside 

pdf 
  2

### Curves only of valid compounds having BMD values of cell count and RF classification

In [ ]:
models <- c("Exp2", "Exp3", "Exp4", "Exp5", "Poly2", "Lin", "Power", "Hill")

ncpus <- 1  

unique_compounds <- unique(df_cc$Metadata_Compound)

feat_cols = colnames(df_cc)[!grepl("Metadata", colnames(df_cc))]


res = list()
gene_table <- list()

all_bmd_pass <- data.frame()

for (compound in unique_compounds) {
  
  compound_data <- df_cc[df_cc$Metadata_Compound %in% c(compound, "DMSO"), ]
  
  dose <- compound_data$Metadata_Concentration
  dose_log <- dose
  dose_log[dose_log > 0] <- log10(dose_log[dose_log > 0])
  
  rank_dose = unique(dose) %>% sort(. , decreasing = TRUE)
  dose_spacing = abs(log10(rank_dose[2]/rank_dose[1]))
  
  dose_shift = abs(min(dose_log)) + dose_spacing
  dose_shifted = dose_log
  dose_shifted[dose_shifted != 0] = dose_shifted[dose_shifted != 0] + dose_shift
  
  if (length(unique(compound_data$Metadata_Concentration)) > 2) {  
    
    print(paste("Processing compound:", compound))  
    dose <- dose_shifted
    
    compound_mat <- t(compound_data[, feat_cols])  
    min_val = abs(min(compound_mat, na.rm=TRUE))  
    add_min = min_val + 0.1 * min_val  
    mat_new = compound_mat + add_min  
    
    if (length(dose) == ncol(mat_new)) {  
      tryCatch({
        fit_obj <- PerformCurveFitting(data = mat_new, dose = dose, ncpus = ncpus, models = models)
        fit_obj <- FilterDRFit(fit_obj, lof.pval = 0.1, filt.var = "AIC.model")
        fit_obj <- PerformBMDCalc(fit_obj, ncpus = ncpus, num.sds = 2, bmr.method = "sample.mean", log10.dose = TRUE)
        
        bmd_res <- fit_obj$bmd_res
        bmd_pass <- bmd_res[bmd_res$all.pass, ]
        fit_obj$bmd_pass <- bmd_pass

        res[[compound]] = fit_obj
        
        if (nrow(bmd_pass) > 0) {
          bmd_pass$compound_name <- compound
          all_bmd_pass <- rbind(all_bmd_pass, bmd_pass) 
        }
      }, error = function(e) {
        print(paste("Error with compound:", compound, "- ignoring this compound"))
      })
    }
  }  
}




In [ ]:
list_plot <- list()

for (i in 1:nrow(all_bmd_pass)) {
  tryCatch({
    gene_id <- all_bmd_pass$gene.id[i]
    compound <- all_bmd_pass$compound_name[i]

    temp <- plot_bmd_curve(gene_id, res[[compound]], return_type = "plot.data")

    temp$protein   <- gene_id
    temp$compound  <- compound
    #temp$category  <-
    temp$bmd       <- all_bmd_pass$bmd[i]
    temp$bmd_l     <- all_bmd_pass$bmdl[i]
    temp$bmd_u     <- all_bmd_pass$bmdu[i]
    
    list_plot[[i]] <- temp
  }, error = function(e) {
    message(sprintf("Erreur pour gene_id %s et compound %s : %s", gene_id, compound, e$message))
  })
}

final_df <- do.call(rbind, list_plot)

print(final_df)

In [33]:
## calculating the bmd values at real concentration

final_df$Concentration <- NA
final_df$Concentration_L <- NA
final_df$Concentration_U <- NA

for (i in 1:nrow(final_df)) {
  compound <- final_df$compound[i]
  bmd <- final_df$bmd[i]
  bmdl <- final_df$bmd_l[i]
  bmdu <- final_df$bmd_u[i]

  compound_data <- df[df$Metadata_Compound == compound, ]
  dose <- compound_data$Metadata_Concentration

  dose_log <- dose
  dose_log[dose_log > 0] <- log10(dose_log[dose_log > 0])

  rank_dose <- unique(dose) %>% sort(decreasing = TRUE)
  dose_spacing <- abs(log10(rank_dose[2] / rank_dose[1]))
  
  dose_shift <- abs(min(dose_log)) + dose_spacing

  final_df$Concentration[i] <- 10^(bmd - dose_shift)
  final_df$Concentration_L[i] <- 10^(bmdl - dose_shift)
  final_df$Concentration_U[i] <- 10^(bmdu - dose_shift)
}

In [ ]:
## we now plot the bmd curves for each compound and protein and save them in a pdf file


library(ggplot2)
library(ggforce)

# Filtrer les données pour le type de protéine "CC0"
final_df_CC0 <- final_df[final_df$protein == "CC0", ]

n_per_page <- 9  

final_df_CC0$compound_protein <- paste(final_df_CC0$compound, final_df_CC0$protein, sep = " - ")

plot_groups_CC0 <- unique(final_df_CC0$compound_protein)
n_pages_CC0 <- ceiling(length(plot_groups_CC0) / n_per_page)

pdf_file_CC0 <- "./outputs/cc0_bmd_curves.pdf"
pdf(pdf_file_CC0, width = 15, height = 10)

for (i in 1:n_pages_CC0) {
  tryCatch({
    p <- ggplot(final_df_CC0, aes(x = x, y = Observations)) +
      geom_point(show.legend = FALSE) +
      geom_line(aes(y = f_x), show.legend = FALSE) +
      geom_vline(aes(xintercept = bmd), linetype = "solid", color = "red") +  
      geom_vline(aes(xintercept = bmd_l), linetype = "dashed", color = "red") + 
      geom_vline(aes(xintercept = bmd_u), linetype = "dashed", color = "red") +  
      facet_wrap_paginate(~ compound_protein, ncol = 3, nrow = 3, page = i, scales = "free_y") +
      labs(title = sprintf("BMD curves - CC0 - Page %d", i),
           x = "Concentration",
           y = "Observation") +
      theme_bw() +
      theme(strip.text = element_text(size = 8))
    
    print(p) 
  }, error = function(e) {
    message(sprintf("Error in the page %d : %s", i, e$message))
  })
}

dev.off()


Warning message:
"Removed 196 rows containing missing values or values outside the scale range
(`geom_point()`)."


pdf 
  2

In [34]:
# curves of probability of Random Forest classification 

final_df_RF_prob <- final_df[final_df$protein == "RF_prob", ]

n_per_page <- 9  

final_df_RF_prob$compound_protein <- paste(final_df_RF_prob$compound, final_df_RF_prob$protein, sep = " - ")

plot_groups_RF_prob <- unique(final_df_RF_prob$compound_protein)
n_pages_RF_prob <- ceiling(length(plot_groups_RF_prob) / n_per_page)

pdf_file_RF_prob <- "./outputs/RF_prob_bmd_curves.pdf"
pdf(pdf_file_RF_prob, width = 15, height = 10)

for (i in 1:n_pages_RF_prob) {
  tryCatch({
    p <- ggplot(final_df_RF_prob, aes(x = x, y = Observations)) +
      geom_point(show.legend = FALSE) +
      geom_line(aes(y = f_x), show.legend = FALSE) +
      geom_vline(aes(xintercept = bmd), linetype = "solid", color = "red") +  
      geom_vline(aes(xintercept = bmd_l), linetype = "dashed", color = "red") + 
      geom_vline(aes(xintercept = bmd_u), linetype = "dashed", color = "red") +  
      facet_wrap_paginate(~ compound_protein, ncol = 3, nrow = 3, page = i, scales = "free_y") +
      labs(title = sprintf("BMD curves - RF_prob - Page %d", i),
           x = "Concentration",
           y = "Observation") +
      theme_bw() +
      theme(strip.text = element_text(size = 8))
    
    print(p) 
  }, error = function(e) {
    message(sprintf("Error in the page %d : %s", i, e$message))
  })
}

dev.off()


Warning message:
"Removed 882 rows containing missing values or values outside the scale range
(`geom_point()`)."


pdf 
  2

In [26]:
## Calculate the mean concentration for each compound


compound_summary <- final_df %>%
  group_by(compound, protein) %>%
  summarise(
    mean_concentration = mean(Concentration, na.rm = TRUE),
    .groups = "drop"
  )

print(compound_summary)


# A tibble: 2 x 3
  compound   protein mean_concentration
  <chr>      <chr>                <dbl>
1 Ethoxyquin CC0                   258.
2 FCCP       CC0                   240.
